In [7]:
from aclg.rules.split.split_ratio import split_by_ratio, SplitOrientation, split_by_ratio_grid
from aclg.rules.split.split_basic import split_horizontal, split_vertical
from aclg.rules.split.split_hold import split_hold
from aclg.rules.spacing import spacing_grid, spacing_vertical, spacing_horizontal
from aclg.post_processing.padding import add_padding
from aclg.rules.symetric.symmetric_1 import split_symmetric_1_horizontal, split_symmetric_1_vertical
from aclg.rules.align import align_components, AlignmentMode
from aclg.dataclass.component import Component
import random
from typing import List, Dict, Any, Tuple

In [8]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from typing import List
from aclg.dataclass.component import Component # 確保 Component 類別已定義或匯入
import random

# 定義一個顏色列表，用於區分不同層級的元件
# 您可以自行增減或修改顏色
LEVEL_COLORS = [
    '#FFB3BA', '#FFDFBA', '#FFFFBA', '#BAFFC9', '#BAE1FF',
    '#E0BBE4', '#FFB6C1', '#FFDAB9', '#E6E6FA', '#B0E0E6'
]

class ComponentPlotter:
    """
    一個用於視覺化 Component 列表的工具，可以遞迴繪製所有子元件。
    """
    def _draw_recursive(self, ax, component: Component):
        """
        [私有輔助函式] 使用遞迴方式繪製單一元件及其所有子元件。
        """
        # 1. 取得元件的繪圖資訊
        top_left_x, top_left_y = component.get_topleft()
        width = component.width
        height = component.height
        level = component.level

        # 2. 根據層級選擇顏色
        color = LEVEL_COLORS[level % len(LEVEL_COLORS)]

        # 3. 建立並新增矩形到圖上
        rect = patches.Rectangle(
            (top_left_x, top_left_y),
            width,
            height,
            linewidth=1.2,
            edgecolor='black',
            facecolor=color,
            alpha=0.8
        )
        ax.add_patch(rect)

        # 4. 在矩形中心加上標籤 (顯示層級和ID)
        label = f"L{level}\nID:{component.relation_id}"
        ax.text(
            component.x,
            component.y,
            label,
            ha='center',
            va='center',
            fontsize=8,
            color='black'
        )
        
        # 5. 遞迴繪製子元件
        if component.sub_components:
            for sub_comp in component.sub_components:
                self._draw_recursive(ax, sub_comp)

    def plot(self, components: List[Component], title: str = "Component Layout"):
        """
        [公開方法] 繪製整個元件列表。
        """
        fig, ax = plt.subplots(1, figsize=(10, 10))
        
        if not components:
            print("Warning: The component list is empty. Nothing to plot.")
            ax.set_title("Empty Component List")
            plt.savefig("empty_plot.png")
            plt.close(fig)
            return

        # 遍歷所有最上層的元件並開始遞迴繪製
        for comp in components:
            self._draw_recursive(ax, comp)
        
        # 自動設定圖表的邊界
        ax.autoscale_view()
        # 設定長寬比為 1:1，確保正方形不會變形成長方形
        ax.set_aspect('equal', adjustable='box')
        
        plt.title(title)
        plt.xlabel("X-axis")
        plt.ylabel("Y-axis")
        plt.grid(True, linestyle='--', alpha=0.6)
        
        # 儲存圖片
        output_filename = "component_visualization.png"
        plt.savefig(output_filename)
        print(f"✅ 繪圖完成！圖片已儲存至 {output_filename}")
        plt.close(fig) # 關閉圖形，釋放記憶體

In [9]:
class Level_0:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.w_range = (100, 120)
        self.h_range = (100, 120)
        self.generate_rule = 'root'
        self.level = 0
        self.relation_id = 0
    
    def generate(self) -> List[Component]:
        return [Component(
            x=self.x,
            y=self.y,
            width=random.randint(*self.w_range),
            height=random.randint(*self.h_range),
            relation_id=self.relation_id,
            generate_rule=self.generate_rule,
            level=self.level
        )]



In [10]:
class Level_1:
    """
    進階版的 Level_1 處理器，修正了對齊邏輯，嚴格遵守分割方向與對齊模式的綁定關係。
    """
    def __init__(
        self,
        w_h_ratio_bound: tuple[float, float] = (1/6, 6/1),
        max_tries_per_orientation: int = 50,
        num_splits_range: tuple[int, int] = (2, 5),
        ratio_range: tuple[float, float] = (0.3, 1.0),
        split_only_probability: float = 0.5,
        align_scale_factor_range: tuple[float, float] = (0.2, 1.0),
        force_align_threshold: int = 3
    ):
        # 儲存所有超參數
        self.w_h_ratio_bound = w_h_ratio_bound
        self.max_tries_per_orientation = max_tries_per_orientation
        self.num_splits_range = num_splits_range
        self.ratio_range = ratio_range
        self.split_only_probability = split_only_probability
        self.align_scale_factor_range = align_scale_factor_range
        self.force_align_threshold = force_align_threshold
        self.level = 1

    # _find_valid_ratios 輔助函式維持不變
    def _find_valid_ratios(self, parent_component: Component, orientation: SplitOrientation, num_splits: int):
        parent_w_h_ratio = parent_component.w_h_ratio()
        min_ratio, max_ratio = self.w_h_ratio_bound
        for _ in range(self.max_tries_per_orientation):
            ratios = [random.uniform(*self.ratio_range) for _ in range(num_splits)]
            total_ratio = sum(ratios)
            all_valid = True
            for r in ratios:
                sub_w_h_ratio = 0
                if orientation == SplitOrientation.HORIZONTAL:
                    sub_w_h_ratio = parent_w_h_ratio * (total_ratio / r)
                else:
                    sub_w_h_ratio = parent_w_h_ratio * (r / total_ratio)
                if not (min_ratio <= sub_w_h_ratio <= max_ratio):
                    all_valid = False
                    break
            if all_valid:
                return ratios
        return None

    def _apply_split(self, parent_component: Component, num_splits: int) -> List[Component]:
        """[行為1] 執行純分割操作"""
        if parent_component.w_h_ratio() > 1:
            orientations_to_try = [SplitOrientation.VERTICAL, SplitOrientation.HORIZONTAL]
        else:
            orientations_to_try = [SplitOrientation.HORIZONTAL, SplitOrientation.VERTICAL]

        for orientation in orientations_to_try:
            valid_ratios = self._find_valid_ratios(parent_component, orientation, num_splits)
            if valid_ratios:
                return split_by_ratio(parent_component, valid_ratios, orientation)

        return split_hold(parent_component)

    def _apply_align(self, parent_component: Component, num_splits: int) -> List[Component]:
        """[行為2] 執行分割後對齊操作 (策略二：使用數學限制法確保縮放合規)"""
        # 1. 隨機選擇對齊模式
        align_mode = random.choice(list(AlignmentMode))
        
        # 2. 根據您修正後的規則，決定分割方向
        if align_mode in [AlignmentMode.TOP, AlignmentMode.BOTTOM, AlignmentMode.CENTER_H]:
            required_orientation = SplitOrientation.VERTICAL
        else:
            required_orientation = SplitOrientation.HORIZONTAL

        # 3. 初始分割
        valid_ratios = self._find_valid_ratios(parent_component, required_orientation, num_splits)
        if not valid_ratios:
            return split_hold(parent_component)
        
        sub_components = split_by_ratio(parent_component, valid_ratios, required_orientation)

        # 4. 為每個子元件計算有效的縮放範圍，並從中生成縮放因子
        scale_factors = []
        min_ratio_bound, max_ratio_bound = self.w_h_ratio_bound
        min_scale_bound, max_scale_bound = self.align_scale_factor_range

        for comp in sub_components:
            original_ratio = comp.w_h_ratio()
            
            if align_mode in [AlignmentMode.TOP, AlignmentMode.BOTTOM, AlignmentMode.CENTER_H]:
                # 改變 height，計算 scale 的有效數學邊界
                valid_min_s = original_ratio / max_ratio_bound
                valid_max_s = original_ratio / min_ratio_bound
            else:
                # 改變 width，計算 scale 的有效數學邊界
                valid_min_s = min_ratio_bound / original_ratio
                valid_max_s = max_ratio_bound / original_ratio

            # 取【數學邊界】和【超參數邊界】的交集，確保縮放不會太誇張
            final_min_s = max(valid_min_s, min_scale_bound)
            final_max_s = min(valid_max_s, max_scale_bound)
            
            # 如果有效範圍不存在，則使用一個安全的預設值
            if final_min_s > final_max_s:
                scale = 1.0 
            else:
                scale = random.uniform(final_min_s, final_max_s)
            
            scale_factors.append(scale)

        # 5. 執行對齊
        return align_components(sub_components, scale_factors, align_mode)

    def _process_single_component(self, parent_component: Component) -> List[Component]:
        """[調度中心] 根據規則決策，並呼叫對應的處理函式"""
        num_splits = random.randint(*self.num_splits_range)
        
        # 1. 優先規則：檢查是否達到強制對齊的門檻
        if num_splits > self.force_align_threshold:
            # print(f"-> 規則觸發：元件數量 {num_splits} > {self.force_align_threshold}，強制對齊。")
            return self._apply_align(parent_component, num_splits)
        
        # 2. 降級規則：如果未達到門檻，則走原本的隨機機率決策
        else:
            if random.random() < self.split_only_probability:
                # print(f"-> 隨機決策：純分割")
                return self._apply_split(parent_component, num_splits)
            else:
                # print(f"-> 隨機決策：分割後對齊")
                return self._apply_align(parent_component, num_splits)

    def generate(self, components: List[Component]) -> List[Component]:
        """[公開方法] 處理元件列表"""
        all_results = []
        relation_id = 0
        for component in components:
            processed_sub_components = self._process_single_component(component)
            for sub_comp in processed_sub_components:
                sub_comp.level = self.level
                sub_comp.relation_id = relation_id
            all_results.extend(processed_sub_components)
            relation_id += 1
            component.sub_components = processed_sub_components
        return all_results

In [11]:
class Level_2:
    """
    Level 2 產生器 (上下文感知版)。

    新版特性：
    1.  **上下文感知**：在選擇對齊模式前，會分析元件在其同級中的相對位置（上/下/左/右），
        並過濾掉會導致元件遠離鄰居的「不合理」對齊選項。
    2.  **修正對齊邏輯**：修正了對齊模式與分割方向的匹配錯誤，確保操作的合理性。
    3.  **保留核心規則**：單次對齊原則、允許間隙等先前版本的核心特性維持不變。
    """
    def __init__(
        self,
        large_component_align_probability: float = 1.0,
        wide_threshold: float = 2.0,
        tall_threshold: float = 0.5,
        size_thresholds: Tuple[float, float] = (0.1, 0.4),
        small_component_hold_probability: float = 0.8,
        policy_wide: Dict[str, Any] = None,
        policy_tall: Dict[str, Any] = None,
        policy_square: Dict[str, Any] = None,
        w_h_ratio_bound: tuple[float, float] = (1/6, 6/1),
        max_tries: int = 50,
        ratio_grid_probability: float = 0.5,
        ratio_range: tuple[float, float] = (0.3, 0.6)
    ):
        # __init__ 內容與之前版本相同
        self.large_component_align_probability = large_component_align_probability
        self.wide_threshold = wide_threshold
        self.tall_threshold = tall_threshold
        self.size_thresholds = size_thresholds
        self.small_component_hold_probability = small_component_hold_probability
        self.policy_wide = policy_wide or {"rows_range": (1, 2), "cols_range": (3, 5),"h_ratios_num_range": (1, 2), "v_ratios_num_range": (3, 5)}
        self.policy_tall = policy_tall or {"rows_range": (3, 5), "cols_range": (1, 2),"h_ratios_num_range": (3, 5), "v_ratios_num_range": (1, 2)}
        self.policy_square = policy_square or {"rows_range": (2, 4), "cols_range": (2, 4),"h_ratios_num_range": (2, 4), "v_ratios_num_range": (2, 4)}
        self.w_h_ratio_bound = w_h_ratio_bound
        self.max_tries = max_tries
        self.ratio_grid_probability = ratio_grid_probability
        self.ratio_range = ratio_range
        self.level = 2

    def _apply_advanced_align(self, parent_component: Component, siblings_bbox: Dict[str, float]) -> List[Component]:
        """[智慧規則] 根據元件在同級中的位置，過濾並選擇合適的對齊模式。"""
        # 1. 根據元件位置，過濾出有效的對齊模式
        valid_align_modes = []
        epsilon = 1e-6 # 用於浮點數比較
        
        # 取得元件的四個邊界
        p_left, p_top = parent_component.get_topleft()
        p_right, p_bottom = parent_component.get_bottomright()

        # 判斷是否靠上/下/左/右邊界
        is_top_edge = abs(p_top - siblings_bbox['min_y']) < epsilon
        is_bottom_edge = abs(p_bottom - siblings_bbox['max_y']) < epsilon
        is_left_edge = abs(p_left - siblings_bbox['min_x']) < epsilon
        is_right_edge = abs(p_right - siblings_bbox['max_x']) < epsilon

        # 根據邊界位置建立合理的對齊選項
        # 如果在頂部，就不能向上對齊(TOP)，應該向下對齊(BOTTOM)
        if is_top_edge: valid_align_modes.append(AlignmentMode.BOTTOM)
        if is_bottom_edge: valid_align_modes.append(AlignmentMode.TOP)
        if is_left_edge: valid_align_modes.append(AlignmentMode.RIGHT)
        if is_right_edge: valid_align_modes.append(AlignmentMode.LEFT)

        # 如果元件在中間（不靠任何邊），則中心對齊是安全的選項
        if not (is_top_edge or is_bottom_edge): valid_align_modes.append(AlignmentMode.CENTER_V)
        if not (is_left_edge or is_right_edge): valid_align_modes.append(AlignmentMode.CENTER_H)
        
        # 如果過濾後沒有任何選項，則使用一個安全的預設值
        if not valid_align_modes:
            align_mode = AlignmentMode.CENTER_H if parent_component.w_h_ratio() <= 1 else AlignmentMode.CENTER_V
        else:
            align_mode = random.choice(valid_align_modes)

        # 2. 【修正】根據選定的對齊模式，決定必須的分割方向
        if align_mode in [AlignmentMode.TOP, AlignmentMode.BOTTOM, AlignmentMode.CENTER_H]:
            orientation = SplitOrientation.VERTICAL
        else: # LEFT, RIGHT, CENTER_V
            orientation = SplitOrientation.HORIZONTAL
        
        num_splits = random.randint(2, 4)
        valid_ratios = [random.uniform(0.5, 1.0) for _ in range(num_splits)]

        # 3. 執行分割與對齊
        sub_components = split_by_ratio(parent_component, valid_ratios, orientation)
        scale_factors = [random.uniform(0.6, 0.95) for _ in range(num_splits)]
        
        return align_components(sub_components, scale_factors, align_mode)

    def _apply_grid_split(self, parent_component: Component, size_ratio: float) -> List[Component]:
        """[標準規則] 為未被選中執行對齊的元件，執行常規的網格分割。"""
        shape_ratio = parent_component.w_h_ratio()
        base_policy = self.policy_square
        if shape_ratio > self.wide_threshold: base_policy = self.policy_wide
        elif shape_ratio < self.tall_threshold: base_policy = self.policy_tall
        final_policy = self._get_dynamic_policy(base_policy, size_ratio)
        if random.random() < self.ratio_grid_probability:
            return self._apply_ratio_grid(parent_component, final_policy)
        else:
            return self._apply_spacing_grid(parent_component, final_policy)

    def generate(self, components: List[Component], root_component: Component) -> List[Component]:
        """[公開方法] 處理整個元件列表，並確保最多只有一個元件被進階對齊。"""
        if not components or not root_component:
            return []
        
        root_area = root_component.width * root_component.height
        
        # --- 新增：計算同級元件的整體邊界框 ---
        min_x = min(c.get_topleft()[0] for c in components)
        max_x = max(c.get_bottomright()[0] for c in components)
        min_y = min(c.get_topleft()[1] for c in components)
        max_y = max(c.get_bottomright()[1] for c in components)
        siblings_bbox = {"min_x": min_x, "max_x": max_x, "min_y": min_y, "max_y": max_y}
        
        # --- 單次對齊原則實現 (不變) ---
        large_thresh = self.size_thresholds[1]
        alignment_candidates = [c for c in components if (c.width * c.height) / root_area > large_thresh]
        component_to_align = None
        if alignment_candidates and random.random() < self.large_component_align_probability:
            component_to_align = random.choice(alignment_candidates)

        # --- 主處理流程 ---
        all_results = []
        relation_id = 0
        for comp in components:
            processed_sub_components = []
            size_ratio = (comp.width * comp.height) / root_area
            small_thresh = self.size_thresholds[0]

            if comp is component_to_align:
                # 傳入邊界框作為上下文
                processed_sub_components = self._apply_advanced_align(comp, siblings_bbox)
            else:
                if size_ratio < small_thresh and random.random() < self.small_component_hold_probability:
                    processed_sub_components = split_hold(comp)
                else:
                    processed_sub_components = self._apply_grid_split(comp, size_ratio)

            for sub_comp in processed_sub_components:
                sub_comp.level = self.level
                sub_comp.relation_id = relation_id
            all_results.extend(processed_sub_components)
            relation_id += 1
            comp.sub_components = processed_sub_components
            
        return all_results

    # (其他輔助函式與之前版本相同，此處省略)
    def _get_dynamic_policy(self, base_policy: Dict[str, Any], size_ratio: float) -> Dict[str, Any]:
        small_thresh, large_thresh = self.size_thresholds; dynamic_policy = base_policy.copy();
        if size_ratio < small_thresh: scale_factor = 0.5 
        elif size_ratio > large_thresh: scale_factor = 1.5 
        else: return dynamic_policy
        for key in ["rows_range", "cols_range", "h_ratios_num_range", "v_ratios_num_range"]:
            min_val, max_val = dynamic_policy[key]; new_min = max(1, int(min_val * scale_factor)); new_max = max(new_min, int(max_val * scale_factor)); dynamic_policy[key] = (new_min, new_max)
        return dynamic_policy
    def _apply_ratio_grid(self, parent_component: Component, policy: Dict[str, Any]) -> List[Component]:
        h_ratios_num_range = policy["h_ratios_num_range"]; v_ratios_num_range = policy["v_ratios_num_range"]
        for _ in range(self.max_tries):
            num_h = random.randint(*h_ratios_num_range); num_v = random.randint(*v_ratios_num_range)
            h_ratios = [random.uniform(*self.ratio_range) for _ in range(num_h)]; v_ratios = [random.uniform(*self.ratio_range) for _ in range(num_v)]
            return split_by_ratio_grid(parent_component, h_ratios, v_ratios)
        return split_hold(parent_component)
    def _apply_spacing_grid(self, parent_component: Component, policy: Dict[str, Any]) -> List[Component]:
        rows_range = policy["rows_range"]; cols_range = policy["cols_range"]
        for _ in range(self.max_tries):
            rows = random.randint(*rows_range); cols = random.randint(*cols_range)
            return spacing_grid(parent_component, rows, cols)
        return split_hold(parent_component)

In [12]:
# --- 1. 初始化所有層級的產生器 ---
level_0_generator = Level_0()
level_1_generator = Level_1()
# 使用新的動態產生器
level_2_generator = Level_2() 

# --- 2. 依序執行生成流程 ---
# Level 0: 產生根元件 (通常只有一個)
root_components = level_0_generator.generate()

# Level 1: 對根元件進行分割
level_1_components = level_1_generator.generate(root_components)

# Level 2: 呼叫時，需要傳入 level_1 的元件 和 level_0 的根元件
# 我們假設 root_components 只有一個元素，所以用 root_components[0]
level_2_components = level_2_generator.generate(level_1_components, root_components[0])

# --- 3. 視覺化最終結果 ---
print(f"根元件尺寸: {root_components[0].width:.1f} x {root_components[0].height:.1f}")
print("-" * 50)
print("Level 1 元件:")
for comp in level_1_components:
    print(f"  - ID {comp.relation_id}, 尺寸: {comp.width:.1f} x {comp.height:.1f}, 長寬比: {comp.w_h_ratio():.2f}")
print("-" * 50)

plotter = ComponentPlotter()
plotter.plot(root_components, title="Hierarchical Layout with Dynamic Level 2")

根元件尺寸: 101.0 x 105.0
--------------------------------------------------
Level 1 元件:
  - ID 0, 尺寸: 25.7 x 83.5, 長寬比: 0.31
  - ID 0, 尺寸: 23.8 x 25.3, 長寬比: 0.94
  - ID 0, 尺寸: 20.3 x 45.4, 長寬比: 0.45
  - ID 0, 尺寸: 31.1 x 59.8, 長寬比: 0.52
--------------------------------------------------
✅ 繪圖完成！圖片已儲存至 component_visualization.png
